# **Are u ready to fine-tune ur Model ?**

* fine-tuning could be done using many techniques like: 

* 1. **Grid Search**: involves searching through a predefined set of hyperparameters to find the best combination that optimizes the model's performance. 
#
* 2. **RandomizedSearchCV**: This function, also from scikit-learn, is similar to GridSearchCV but instead of exhaustively searching through all possible combinations of hyperparameters, it samples a fixed number of hyperparameter settings from specified distributions. This can be more efficient when the hyperparameter space is large.
#
* 3. **Bayesian Optimization**: Bayesian optimization is a more sophisticated approach that uses probabilistic models to predict which hyperparameters are likely to perform well. Libraries like scikit-optimize and hyperopt provide implementations of Bayesian optimization.
#
* 4. **Cross-Validation Strategies**: Instead of standard k-fold cross-validation, you can use more advanced strategies like stratified k-fold, time series cross-validation, or leave-one-out cross-validation depending on your data's characteristics.
#
* 5. **Early Stopping**: Instead of running a fixed number of epochs, use early stopping to monitor the model's performance on a validation set and stop training once performance plateaus or starts to degrade.
#
* 6. **Learning Rate Schedulers**: When training neural networks, learning rate schedulers adjust the learning rate over time to help the model converge more efficiently. Examples include ReduceLROnPlateau and LearningRateScheduler in Keras.
#
* 7. ***Ensemble Methods**: Combine multiple models to create an ensemble that often performs better than individual models. Techniques like bagging (Bootstrap Aggregating) and boosting (AdaBoost, XGBoost, LightGBM) can be used.
#
* 9. **Regularization**: Techniques like L1 and L2 regularization can help prevent overfitting by adding penalty terms to the loss function based on the magnitudes of model parameters.
#
* 10. **Data Augmentation**: In computer vision tasks, data augmentation techniques such as rotation, flipping, and cropping can artificially increase the size of your training dataset, leading to improved generalization.
#
* 11. **Hyperparameter Libraries**: Libraries like optuna and hyperopt offer more advanced hyperparameter tuning strategies beyond grid and random search, such as Tree-structured Parzen Estimators (TPE) and Gaussian Processes.
#
* 12. **Visualization and Analysis Tools**: Tools like scikit-learn's validation_curve and learning_curve functions can help you visualize how changing specific hyperparameters or dataset sizes affects model performance.
#
* 13. **Automated Machine Learning (AutoML)**: Platforms like Auto-sklearn, H2O.ai, and Google AutoML automate the process of hyperparameter tuning and model selection.
#

## **Grid Search**

In [2]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def grid_search(X_train, y_train, model, grid_params, cv):
    """
    Perform grid search for hyperparameter tuning using cross-validation.

    Parameters:
        X_train (array-like): Training input features.
        y_train (array-like): Training target labels.
        model: The machine learning model to be tuned.
        grid_params (dict) or (list of dicts): Hyperparameter grid to search.
        cv: Number of cross-validation folds.

    Returns:
        best_estimator: The best estimator (model) found during grid search.
    """
    gs = GridSearchCV(estimator=model, param_grid=grid_params, cv=cv, return_train_score=True, scoring='neg_mean_squared_error')
    gs.fit(X_train, y_train)
    print("Best Params:", gs.best_params_)
    print("Best Estimator:", gs.best_estimator_)

    # Print the results
    cvres = gs.cv_results_
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(np.sqrt(-mean_score), params)

    return gs.best_estimator_

# --------------------------------------- usage example --------------------------------------------

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a SVM model
svm_model = SVC()

# Define hyperparameters and their possible values for grid search
grid_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.1, 1, 'scale']
}

# Perform grid search
best_model = grid_search(X_train, y_train, svm_model, grid_params, cv=5)

# Fit the best model on the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


Best Params: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Best Estimator: SVC(C=1, gamma=0.1, kernel='linear')
0.24152294576982397 {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
0.31622776601683794 {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
0.24152294576982397 {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
0.24152294576982397 {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
0.24152294576982397 {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
0.32914029430219166 {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
0.2041241452319315 {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
0.22360679774997896 {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.2041241452319315 {'C': 1, 'gamma': 1, 'kernel': 'linear'}
0.2041241452319315 {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
0.2041241452319315 {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
0.22360679774997896 {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.22360679774997896 {'C': 10, 'gamma': 0.1, 'kernel': 'linear'}
0.22360679774997896 {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.2236067977

## **Randomized Search**

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

def randomized_search(X_train, y_train, model, param_dist, cv, n_iter=10):
    """
    Perform randomized search for hyperparameter tuning using cross-validation.

    Parameters:
        X_train (array-like): Training input features.
        y_train (array-like): Training target labels.
        model: The machine learning model to be tuned.
        param_dist (dict): Hyperparameter distributions to sample from.
        cv: Number of cross-validation folds.
        n_iter: Number of parameter settings that are sampled.

    Returns:
        best_estimator: The best estimator (model) found during randomized search.
    """
    random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=n_iter, cv=cv, return_train_score=True, scoring='neg_mean_squared_error')
    random_search.fit(X_train, y_train)
    print("Best Params:", random_search.best_params_)
    print("Best Estimator:", random_search.best_estimator_)

    # Print the results
    cvres = random_search.cv_results_
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(np.sqrt(-mean_score), params)

    return random_search.best_estimator_


# --------------------------------------- usage example --------------------------------------------

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a SVM model
svm_model = SVC()

# Define hyperparameters and their distributions for randomized search
param_dist = {
    'C': np.logspace(-3, 2, 6),  # Sample values from a logarithmic range between 10^-3 and 10^2
    'kernel': ['linear', 'rbf'],  # Sample values from these kernel options
    'gamma': [0.1, 1, 'scale']  # Sample values from these specific gamma values
}


# Perform randomized search
best_model = randomized_search(X_train, y_train, svm_model, param_dist, cv=5, n_iter=10)

# Fit the best model on the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


Best Params: {'kernel': 'linear', 'gamma': 'scale', 'C': 1.0}
Best Estimator: SVC(kernel='linear')
0.7359800721939872 {'kernel': 'linear', 'gamma': 0.1, 'C': 0.001}
0.22360679774997896 {'kernel': 'linear', 'gamma': 'scale', 'C': 10.0}
0.24152294576982397 {'kernel': 'rbf', 'gamma': 1, 'C': 0.1}
0.24152294576982397 {'kernel': 'linear', 'gamma': 'scale', 'C': 0.1}
0.2041241452319315 {'kernel': 'linear', 'gamma': 'scale', 'C': 1.0}
0.22360679774997896 {'kernel': 'rbf', 'gamma': 0.1, 'C': 10.0}
0.22360679774997896 {'kernel': 'linear', 'gamma': 0.1, 'C': 100.0}
0.32914029430219166 {'kernel': 'linear', 'gamma': 'scale', 'C': 0.01}
0.22360679774997896 {'kernel': 'rbf', 'gamma': 'scale', 'C': 1.0}
0.24152294576982397 {'kernel': 'rbf', 'gamma': 'scale', 'C': 100.0}
Test Accuracy: 1.0
